# Importing Library

In [1]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import sys

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('HousingData.csv')
df.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7
6,0.08829,12.5,7.87,NaN,0.524,6.012,66.6,5.5605,5,311,15.2,395.60,12.43,22.9
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15,27.1
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,16.5
9,0.17004,12.5,7.87,NaN,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10,18.9


In [3]:
df.dropna(inplace=True)

In [4]:
X = df.iloc[:,:-1]

y = df['MEDV']

columns = X.columns


In [5]:
sceler = StandardScaler()
X = sceler.fit_transform(X)
X = pd.DataFrame(X, columns= columns)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Reset indices for both X_train and y_train
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [7]:
print('X_train:',x_train.shape)
print('y_train:',y_train.shape)
print('X_test:',x_test.shape)
print('y_test:',y_test.shape)

X_train: (315, 13)
y_train: (315,)
X_test: (79, 13)
y_test: (79,)


In [8]:
import torch
from torch import nn

class Data(torch.utils.data.Dataset):
    def __init__(self,X,y):
        super().__init__()
        self.X = torch.from_numpy(X.values)
        self.y = torch.from_numpy(y.values)
        
    def __len__(self): 
        return len(self.X)
    
    def __getitem__(self,i): 
        return (self.X[i], self.y[i])

class Neural_Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(13,32),
            nn.Sigmoid(),
            nn.Linear(32,1)
        )
        
    def forward(self, X):
        return self.layer(X)

NN = Neural_Network()

train_dataset = Data(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8)

optimizer = torch.optim.SGD(NN.parameters(), lr=1e-4)

loss_func = nn.MSELoss()

epochs = 1000

for epoch in range(epochs):
    total_training_loss = 0
    for i, data in enumerate(train_loader,0):
        features,target = data
        features,target = features.float(),target.float()
        target = target.reshape(-1,1)
        
        optimizer.zero_grad()
        
        pred = NN.forward(features)

        loss = loss_func(pred, target)
        total_training_loss += loss.item()

        loss.backward() 
        optimizer.step()
    if epoch % 50 == 0:
        print(f'epoch: {epoch} - loss: {total_training_loss}')

epoch: 0 - loss: 21763.984970092773
epoch: 50 - loss: 1191.1465878486633
epoch: 100 - loss: 947.0723602771759
epoch: 150 - loss: 877.1032636165619
epoch: 200 - loss: 845.6210987567902
epoch: 250 - loss: 825.71253490448
epoch: 300 - loss: 810.3089337348938
epoch: 350 - loss: 797.1347894668579
epoch: 400 - loss: 785.3175086975098
epoch: 450 - loss: 774.4529523849487
epoch: 500 - loss: 764.3110189437866
epoch: 550 - loss: 754.7357664108276
epoch: 600 - loss: 745.6100084781647
epoch: 650 - loss: 736.8406665325165
epoch: 700 - loss: 728.3526933193207
epoch: 750 - loss: 720.0856139659882
epoch: 800 - loss: 711.9904658794403
epoch: 850 - loss: 704.0291895866394
epoch: 900 - loss: 696.1729097366333
epoch: 950 - loss: 688.400639295578


In [9]:
predicts = NN.forward(torch.from_numpy(x_test.values).float())

In [10]:
predicts = predicts.detach().numpy().reshape(-1,)

In [11]:
res = pd.DataFrame()
res["Actual"] = y_test.values
res["predict"] = predicts
res

,Actual,predict
0,30.1,33.894901
1,15.1,15.376372
2,50.0,39.236214
3,24.0,30.314793
4,19.9,18.047581
...,...,...
74,23.0,20.344608
75,18.3,18.163086
76,35.1,35.073322
77,23.2,21.870560
